In [1]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser, RegexParser
from pydantic import BaseModel, Field
# from langchain_community.document_loaders import PyPDFLoader
from typing import List, Dict, Any
import json
import torch
import re

from knowledgegraph import KnowledgeGraph

In [2]:
with open("hf.key") as f:
    hf_token = f.read()

In [3]:
BYTES_IN_GB = 1000_000_000

def print_mem(msg = ""):
    (free, total) = torch.cuda.mem_get_info()
    used = total - free
    
    perc_usaged = round(used / total * 100.0, 1)
    used_gb = round(used / BYTES_IN_GB, 1)
    total_gb = round(total / BYTES_IN_GB, 1)
    print(f'CUDA mem usage: {used_gb}/{total_gb}GB ({perc_usaged}%)')

print_mem()

CUDA mem usage: 1.1/12.5GB (9.2%)


In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

# MODEL_NAME = "Trelis/Mistral-7B-Instruct-v0.1-Summarize-16k"
MODEL_NAME= "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    token=hf_token
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=hf_token
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.1,
    repetition_penalty=1.2
)

llm = HuggingFacePipeline(pipeline=pipe)

print_mem()

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CUDA mem usage: 9.2/12.5GB (73.5%)


In [5]:
class People(BaseModel):
    people: List[str]

class Relation(BaseModel):
    source: str = Field(description="Name of the source Person")
    target: str = Field(description="Name of the target Person")
    relations: str = Field(description="Nature of the relationship. E.g.: Friend, Foe, Family.")
    
class Relations(BaseModel):
    relations: List[Relation]

people_parser = PydanticOutputParser(pydantic_object=People)

rel_parser = PydanticOutputParser(pydantic_object=Relations)

print(people_parser.get_format_instructions())
print(rel_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"people": {"items": {"type": "string"}, "title": "People", "type": "array"}}, "required": ["people"]}
```
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is

In [6]:
class KnowledgeGraphLLM:
    def __init__(self):
        self.kg = KnowledgeGraph()
        self.kg.clear()
        self.llm = llm
        
        # Define prompts for different operations
        self.people_prompt = PromptTemplate(
            input_variables=["context"],
            partial_variables={
                "format_instructions": people_parser.get_format_instructions()
            },
            template="""[INST] Read the given context and identify any people.
            Keep your reasoning short and concise. If you don't know the answer, return an empty JSON object instead.

            [CONTEXT]
            {context}
            [/CONTEXT]

            [FORMAT]
            {format_instructions}
            [/FORMAT]
            
            [/INST]""")
        
        self.relation_prompt = PromptTemplate(
            input_variables=["context"],
            partial_variables={
                "format_instructions": rel_parser.get_format_instructions()
            },
            template="""[INST] Read the given context and identify any relations using only the provided knowledge graph data.
            Keep your reasoning short and concise. If you don't know the answer, return an empty JSON object instead.

            [GRAPH DATA]
            {graph_data}
            [/GRAPH_DATA]
        
            [CONTEXT]
            {context}
            [/CONTEXT]

            [FORMAT]
            {format_instructions}
            [/FORMAT]
        
            [/INST]""")
        
        self.query_prompt = PromptTemplate(
            input_variables=["question", "graph_data"],
            template="""[INST] Answer the following question using only the provided knowledge graph data.
            If you cannot answer with certainty, say "I cannot determine this from the available data."
            
            [GRAPH DATA]
            {graph_data}
            [/GRAPH_DATA]
            
            [QUESTION]
            {question}
            [/QUESTION]
            
            [/INST]""")
        
        # Create LangChain chains
        self.people_chain = self.people_prompt | self.llm
        self.relation_chain = self.relation_prompt | self.llm
        self.query_chain = self.query_prompt | self.llm

    def extract_people(self, subject: str, context):
        try:
            output = self.people_chain.invoke(input={
              "context": context
            })

            result = re.findall(r'```json(.*?)```', output, re.DOTALL)[-1]

            people = json.loads(result.strip())["people"]
            for person in people:
                self.kg.add_node(person)

            return people
            
        except Exception as e:
            print("Error: Could not parse LLM response as JSON\n")
            print(e, '\n\n')
            print('>', output, '<')
            return []

    def extract_relationships(self, subject: str, context: str) -> List[Dict[str, str]]:
        """Extract relationships from unstructured text using LLM."""
        output = ""
        try:
            # Get LLM's analysis
            output = self.relation_chain.invoke(input={
                "context": context, 
                "graph_data": self.graph_data()
            })

            result = re.findall(r'```json(.*?)```', output, re.DOTALL)[-1]
            
            relationships = json.loads(result.strip())["relations"]
            
            # Add all extracted relationships to the knowledge graph
            for rel in relationships:
                self.kg.add_node(rel["source"])
                self.kg.add_node(rel["target"])
                self.kg.add_edge(rel["source"], rel["target"], rel["relation"])
            
            return relationships
        
        except Exception as e:
            print("Error: Could not parse LLM response as JSON\n")
            print(e, '\n\n')
            print('>', output, '<')
            return []

    def graph_data(self):
        """Dump graph data to JSON"""
        return json.dumps(self.kg.dump(), indent=2)
        
    def smart_query(self, question: str) -> str:
        """Query the knowledge graph using LLM-powered reasoning."""
        # Get LLM's analysis
        return self.query_chain.invoke(input={
            "question": question,
            "graph_data": self.graph_data()
        })
    
    def get_graph_summary(self) -> Dict[str, Any]:
        """Return a summary of the knowledge graph."""
        return {
            "node_count": len(self.kg.nodes),
            "edge_count": sum(len(edges) for edges in self.kg.edges.values()),
            "data": self.kg.dump()
        }

# Example usage
def query_llm(kg_llm, contexts, subject, questions):
    # Add information through unstructured text
    # Extract and add relationships

    for i, ctx in enumerate(contexts):
        people = kg_llm.extract_people(subject, ctx)
    #     relationships = kg_llm.extract_relationships(subject, ctx)
    #     print(f"\r\nExtracted data for ctx#{i}\n{ctx[:50]}:\n")
    #     print(json.dumps({"people": people, "relationships": relationships}, indent=2))
    
    # # Query the enhanced knowledge graph
    # for question in questions:
    #     answer = kg_llm.smart_query(question).partition("[/INST]")[-1]
    #     print(f"###\n\nQ: {question}\nA: {answer}\n\n###")
    
    # Get graph summary
    print("\nKnowledge Graph Summary:")
    print(json.dumps(kg_llm.get_graph_summary(), indent=2))

In [7]:
snippets = [
"""
This is written in the train from Varna to Galatz. Last
night we all assembled a little before the time of sunset. Each of us
had done his work as well as he could; so far as thought, and endeavour,
and opportunity go, we are prepared for the whole of our journey, and
for our work when we get to Galatz. When the usual time came round Mrs.
Harker prepared herself for her hypnotic effort; and after a longer and
more serious effort on the part of Van Helsing than has been usually
necessary, she sank into the trance. Usually she speaks on a hint; but
this time the Professor had to ask her questions, and to ask them pretty
resolutely, before we could learn anything; at last her answer came:--
""",
"""
“I can see nothing; we are still; there are no waves lapping, but only a
steady swirl of water softly running against the hawser. I can hear
men’s voices calling, near and far, and the roll and creak of oars in
the rowlocks. A gun is fired somewhere; the echo of it seems far away.
There is tramping of feet overhead, and ropes and chains are dragged
along. What is this? There is a gleam of light; I can feel the air
blowing upon me.”
""",
"""
Here she stopped. She had risen, as if impulsively, from where she lay
on the sofa, and raised both her hands, palms upwards, as if lifting a
weight. Van Helsing and I looked at each other with understanding.
Quincey raised his eyebrows slightly and looked at her intently, whilst
Harker’s hand instinctively closed round the hilt of his Kukri. There
was a long pause. We all knew that the time when she could speak was
passing; but we felt that it was useless to say anything. Suddenly she
sat up, and, as she opened her eyes, said sweetly:--
""",
"""
“Would none of you like a cup of tea? You must all be so tired!” We
could only make her happy, and so acquiesced. She bustled off to get
tea; when she had gone Van Helsing said:--
""",
"""
“You see, my friends. _He_ is close to land: he has left his
earth-chest. But he has yet to get on shore. In the night he may lie
hidden somewhere; but if he be not carried on shore, or if the ship do
not touch it, he cannot achieve the land. In such case he can, if it be
in the night, change his form and can jump or fly on shore, as he did
at Whitby. But if the day come before he get on shore, then, unless he
be carried he cannot escape. And if he be carried, then the customs men
may discover what the box contain. Thus, in fine, if he escape not on
shore to-night, or before dawn, there will be the whole day lost to him.
We may then arrive in time; for if he escape not at night we shall come
on him in daytime, boxed up and at our mercy; for he dare not be his
true self, awake and visible, lest he be discovered.”
"""
]
questions = []

query_llm(KnowledgeGraphLLM(), snippets, "Dracula", questions)


Knowledge Graph Summary:
{
  "node_count": 10,
  "edge_count": 0,
  "data": {
    "people": {
      "Mrs. Harker": {},
      "Van Hellingen": {},
      "seafarers": {},
      "sailors": {},
      "Quinci": {},
      "Harker": {},
      "She": {},
      "my friends": {},
      "he": {},
      "customs men": {}
    },
    "relations": {
      "Mrs. Harker": {},
      "Van Hellingen": {},
      "seafarers": {},
      "sailors": {},
      "Quinci": {},
      "Harker": {},
      "She": {},
      "my friends": {},
      "he": {},
      "customs men": {}
    }
  }
}


In [7]:
print_mem()

CUDA mem usage: 9.2/12.5GB (73.5%)


In [8]:
def parse_book():
    # Initialize the system
    kg_llm = KnowledgeGraphLLM()
    
    with open("data/dracula.txt") as f:
        context = f.read()
        context = context.rpartition(r"\n\nDRACULA\n\n")[-1]

    paragraphs = []
    chapters = re.split(r"\n\nCHAPTER\ .*\n", context)
    for c in chapters:
        for p in re.split(r"\n\n", c):
            paragraphs.append(p)


    questions = [
        "Who are the main characters in this story?",
        "What is their relation to each other?",
        "Who is the antagonist of the story?",
        "How does the antagonist end up?"
    ]
    
    query_llm(kg_llm, paragraphs, "Dracula", questions)

print_mem()

CUDA mem usage: 9.2/12.5GB (73.5%)
